### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [106]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

from time import time
import warnings

warnings.filterwarnings("ignore")

In [34]:
df = pd.read_csv("./data/dataset-minimal.csv")

In [35]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


In [36]:
df["humor"].value_counts(normalize=True)

humor
False    0.500215
True     0.499785
Name: proportion, dtype: float64

#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

Pre-processing

In [101]:
X = df["text"]

In [102]:
Xt = X.apply(nltk.word_tokenize)

In [89]:
stemmer = PorterStemmer()
Xs = Xt.apply(lambda x: [stemmer.stem(xk) for xk in x]).apply(lambda x: " ".join(x))

In [107]:
lemma = WordNetLemmatizer()
Xl = Xt.apply(lambda x: [lemma.lemmatize(xk) for xk in x]).apply(lambda x: " ".join(x))

In [56]:
y = df["humor"]

In [92]:
def make_pipeline(vectorizer_name: str, classifier_name: str):
    return Pipeline(
        [
            ("vectorizer", eval(vectorizer_name)()),
            ("classifier", eval(classifier_name)()),
        ]
    )

In [133]:
preprocessors = ["none", "stemmer", "lemmatizer"]
vectorizers = ["CountVectorizer", "TfidfVectorizer"]
classifiers = ["LogisticRegression", "DecisionTreeClassifier", "MultinomialNB"]

In [122]:
all_param_grids = {
    "CountVectorizer": {
        "vectorizer__stop_words": [None, "english"],
    },
    "TfidfVectorizer": {
        "vectorizer__stop_words": [None, "english"],
    },
}

In [135]:
results_list = []

for preprocessor in preprocessors:
    if preprocessor == "none":
        Xp = X
    elif preprocessor == "stemmer":
        Xp = Xs
    elif preprocessor == "lemmatizer":
        Xp = Xl
    else:
        raise ValueError(f"Invalid choice '{preprocessor}' for preprocessor")

    X_train, X_test, y_train, y_test = train_test_split(
        Xp, y, random_state=42, test_size=0.25
    )

    for vectorizer in vectorizers:
        for classifier in classifiers:
            print(
                f"Running preprocessor = {preprocessor}, vectorizer = {vectorizer}, and classifier = {classifier}"
            )
            tic = time()
            grid = GridSearchCV(
                estimator=make_pipeline(
                    vectorizer_name=vectorizer, classifier_name=classifier
                ),
                param_grid=all_param_grids[vectorizer],
            ).fit(X_train, y_train)
            grid_search_time = time() - tic
            results_list.append(
                {
                    "preprocessor": preprocessor,
                    "vectorizer": vectorizer,
                    "classifier": classifier,
                    "average fit time": grid_search_time
                    / len(grid.cv_results_["params"]),
                    "best score": grid.best_estimator_.score(X_test, y_test),
                    "best params": grid.best_params_,
                }
            )

Running preprocessor = none, vectorizer = CountVectorizer, and classifier = LogisticRegression
Running preprocessor = none, vectorizer = CountVectorizer, and classifier = DecisionTreeClassifier
Running preprocessor = none, vectorizer = CountVectorizer, and classifier = MultinomialNB
Running preprocessor = none, vectorizer = TfidfVectorizer, and classifier = LogisticRegression
Running preprocessor = none, vectorizer = TfidfVectorizer, and classifier = DecisionTreeClassifier
Running preprocessor = none, vectorizer = TfidfVectorizer, and classifier = MultinomialNB
Running preprocessor = stemmer, vectorizer = CountVectorizer, and classifier = LogisticRegression
Running preprocessor = stemmer, vectorizer = CountVectorizer, and classifier = DecisionTreeClassifier
Running preprocessor = stemmer, vectorizer = CountVectorizer, and classifier = MultinomialNB
Running preprocessor = stemmer, vectorizer = TfidfVectorizer, and classifier = LogisticRegression
Running preprocessor = stemmer, vectorize

In [138]:
pd.DataFrame(results_list).set_index("classifier").sort_values(
    by="best score", ascending=False
)

,preprocessor,vectorizer,average fit time,best score,best params
classifier,,,,,
LogisticRegression,none,CountVectorizer,12.796625,0.92400,{'vectorizer__stop_words': None}
LogisticRegression,lemmatizer,CountVectorizer,11.746000,0.92340,{'vectorizer__stop_words': None}
LogisticRegression,stemmer,CountVectorizer,10.387622,0.92064,{'vectorizer__stop_words': None}
LogisticRegression,none,TfidfVectorizer,7.085414,0.91676,{'vectorizer__stop_words': None}
LogisticRegression,lemmatizer,TfidfVectorizer,7.762917,0.91348,{'vectorizer__stop_words': None}
MultinomialNB,none,CountVectorizer,3.961107,0.91276,{'vectorizer__stop_words': None}
LogisticRegression,stemmer,TfidfVectorizer,7.584452,0.91132,{'vectorizer__stop_words': None}
MultinomialNB,lemmatizer,CountVectorizer,3.987883,0.91004,{'vectorizer__stop_words': None}
MultinomialNB,none,TfidfVectorizer,4.115337,0.90980,{'vectorizer__stop_words': None}


## Fully standalone snippet

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv("./data/dataset-minimal.csv")

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["humor"], random_state=42
)

# All defaults
estimator = Pipeline(
    [
        ("vectorizer", CountVectorizer()),
        ("classifier", MultinomialNB()),
    ]
).fit(X_train, y_train)

acc_all_default = estimator.score(X_test, y_test)

# Grid over a single parameter - stop words, no stop words
grid = GridSearchCV(
    estimator=estimator,
    param_grid={
        "vectorizer__stop_words": [None, "english"],
    },
).fit(X_train, y_train)

acc_grid = grid.best_estimator_.score(X_test, y_test)

[
    acc_all_default,
    acc_grid,
]

[0.91276, 0.91276]